In [6]:
import numpy as np
from typing import Callable, List, Union

def penalty_method(
    f: Callable[[np.ndarray], float],
    grad_f: Callable[[np.ndarray], np.ndarray],
    inequality_constraints: List[Callable[[np.ndarray], float]],
    grad_inequality: List[Callable[[np.ndarray], np.ndarray]],
    x0: np.ndarray,
    k0: float = 1.0,
    alpha: float = 0.01,
    max_iter: int = 100,
    tol: float = 1e-5,
    penalty_type: str = "quadratic",
    increase_k: Callable[[float], float] = lambda k: 2 * k,
) -> np.ndarray:
    """
    Параметры:
        f: Целевая функция.
        grad_f: Градиент целевой функции.
        inequality_constraints: Список функций ограничений g_i(x) >= 0.
        grad_inequality: Список градиентов ограничений.
        x0: Начальная точка.
        k0: Начальный коэффициент штрафа.
        alpha: Шаг градиентного спуска.
        max_iter: Максимальное число итераций.
        tol: Допустимое нарушение ограничений.
        increase_k: Функция увеличения коэффициента штрафа (по умолчанию k *= 2).
    """
    def penalty(x: np.ndarray, k: float) -> float:
        """Штрафная функция."""
        penalty_value = 0.0
        for g in inequality_constraints:
            violation = max(0, -g(x))
            if penalty_type == "quadratic":
                penalty_value += violation ** 2
            elif penalty_type == "linear":
                penalty_value += violation
            elif penalty_type == "barrier":
                if violation <= 0:
                    penalty_value += -np.log(g(x))
                else:
                    penalty_value += np.inf
        return f(x) + k * penalty_value

    def grad_penalty(x: np.ndarray, k: float) -> np.ndarray:
        """Градиент штрафной функции."""
        grad = grad_f(x)
        for i, (g, grad_g) in enumerate(zip(inequality_constraints, grad_inequality)):
            violation = max(0, -g(x))
            if penalty_type == "quadratic":
                if violation > 0:
                    grad += k * (-2 * violation) * grad_g(x)
            elif penalty_type == "linear":
                if violation > 0:
                    grad += k * (-1) * grad_g(x)
            elif penalty_type == "barrier":
                if g(x) <= 0:
                    grad += np.inf * np.ones_like(x)
                else:
                    grad += k * (-1 / g(x)) * grad_g(x)
        return grad

    x = x0.copy()
    k = k0

    for _ in range(max_iter):
        # Градиентный спуск для текущего k
        for _ in range(100):  # Внутренний цикл градиентного спуска
            grad = grad_penalty(x, k)
            x_new = x - alpha * grad
            if np.linalg.norm(grad) < 1e-6:
                break
            x = x_new

        # Проверка ограничений
        violations = [max(0, -g(x)) for g in inequality_constraints]
        if all(v <= tol for v in violations):
            break

        # Увеличиваем коэффициент штрафа
        k = increase_k(k)

    return x

In [ ]:
def f(x):
    return 4 * x[0] - x[1]**2 - 12

def grad_f(x):
    return np.array([4, -2 * x[1]])

def g1(x):
    return 10*x[0] - x[0]**2 + 10*x[1] - x[1]**2 - 34

def grad_g1(x):
    return np.array([10 - 2*x[0], 10 - 2*x[1]])

def g2(x):
    return x[0]

def grad_g2(x):
    return np.array([1, 0])

def g3(x):
    return x[1]

def grad_g3(x):
    return np.array([0, 1])

x0 = np.array([2.0, 4.0])

solution = penalty_method(
    f=f,
    grad_f=grad_f,
    inequality_constraints=[g1, g2, g3],
    grad_inequality=[grad_g1, grad_g2, grad_g3],
    x0=x0,
    penalty_type="linear",
)

print(f"Решение: {solution}")
print(f"f(x) = {f(solution)}")
print(f"Ограничения: g1(x)={g1(solution)}, g2(x)={g2(solution)}, g3(x)={g3(solution)}")

Решение: [4.14813519 8.81016194]
f(x) = -73.02641271787296
Ограничения: g1(x)=0.7569923066608197, g2(x)=4.148135190885445, g3(x)=8.810161944108334
